# Install Libraries

In [ ]:
%pip install segment-geospatial groundingdino-py leafmap localtileserver
%pip install -q git+https://github.com/huggingface/transformers.git

# Import Dependencies

In [ ]:
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm
import torch
from PIL import Image
import io
import leafmap
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM

In [ ]:
sidewalk_test = load_dataset("back2classroom/sidewalks", split='val', streaming=True)